In [1]:
import sys, subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'gspread'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'oauth2client'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'slackbot'])

0

In [ ]:
import pandas as pd
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from slackbot.bot import Bot

In [57]:
def auth():
    SP_CREDENTTIAL_FILE='secret.json'
    SP_SCOPE=[
            'https://spreadsheets.google.com/feeds',
            'https://www.googleapis.com/auth/drive'
    ]
    SP_SHEET_KEY = '17daAsaIsevTmLBH7HLqG4HWMIJi7vVqUEQ6JNCGjKKk'
    SP_SHEET = 'timesheet'
    credentials =ServiceAccountCredentials.from_json_keyfile_name(SP_CREDENTTIAL_FILE,SP_SCOPE)
    gc = gspread.authorize(credentials)

    worksheet= gc.open_by_key(SP_SHEET_KEY).worksheet(SP_SHEET)
    return worksheet

In [58]:
df=pd.DataFrame(worksheet.get_all_records())
df

,日付,出勤時間,退勤時間
0,2021/03/10,10:00,11:00


In [59]:
timestamp =datetime.now()
date=timestamp.strftime('%Y/%m/%d')
date

'2021/03/31'

In [60]:
punch_in =timestamp.strftime('%H:%M')
punch_in

'11:43'

In [61]:
df=df.append({'日付':date,'出勤時間':punch_in,'退勤時間':'なし'},ignore_index=True)

In [62]:
worksheet.update([df.columns.values.tolist()]+df.values.tolist())

{'spreadsheetId': '17daAsaIsevTmLBH7HLqG4HWMIJi7vVqUEQ6JNCGjKKk',
 'updatedRange': 'timesheet!A1:C3',
 'updatedRows': 3,
 'updatedColumns': 3,
 'updatedCells': 9}

In [64]:
punch_out=timestamp.strftime('%H:%M')
punch_out

'11:43'

In [65]:
df.iloc[-1,2]=punch_out

In [66]:
df

,日付,出勤時間,退勤時間
0,2021/03/10,10:00,11:00
1,2021/03/31,11:43,11:43


In [ ]:
worksheet.update([df.columns.values.tolist()]+df.values.tolist())

In [17]:
import sys, subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'gspread'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'oauth2client'])

import pandas as pd
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def auth():
    SP_CREDENTTIAL_FILE='secret.json'
    SP_SCOPE=[
            'https://spreadsheets.google.com/feeds',
            'https://www.googleapis.com/auth/drive'
    ]
    SP_SHEET_KEY = '17daAsaIsevTmLBH7HLqG4HWMIJi7vVqUEQ6JNCGjKKk'
    SP_SHEET = 'timesheet'
    credentials =ServiceAccountCredentials.from_json_keyfile_name(SP_CREDENTTIAL_FILE,SP_SCOPE)
    gc = gspread.authorize(credentials)

    worksheet= gc.open_by_key(SP_SHEET_KEY).worksheet(SP_SHEET)
    return worksheet
#出勤
def punch_in():
    worksheet=auth()
    df=pd.DataFrame(worksheet.get_all_records())
    timestamp =datetime.now()
    date=timestamp.strftime('%Y/%m/%d')
    punch_in =timestamp.strftime('%H:%M')
    df=df.append({'日付':date,'出勤時間':punch_in,'退勤時間':'なし'},ignore_index=True)
    worksheet.update([df.columns.values.tolist()]+df.values.tolist())
    print("出勤完了しました")
    return(punch_in)

def punch_out():
    worksheet=auth()
    df=pd.DataFrame(worksheet.get_all_records())
    timestamp =datetime.now()
    punch_out =timestamp.strftime('%H:%M')
    df.iloc[-1,2]=punch_out
    worksheet.update([df.columns.values.tolist()]+df.values.tolist())
    print("退勤完了しました")
    return(punch_out)
    

In [72]:
punch_in()


出勤完了しました


In [76]:
punch_out()

退勤完了しました


In [ ]:
bot=Bot()
bot.run()

In [18]:
from slackbot.bot import respond_to
from slackbot.bot import listen_to

In [19]:
@respond_to('出勤')
def sample(massage):
    massage.send(punch_in())
@respond_to('退勤')
def sample1(message):
    message.reply(punch_out())

In [ ]:
bot.run()

出勤完了しました
退勤完了しました
